In [1]:
import requests as r
import pandas as pd
import numpy as np
import os

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import pandas_utils as p
import google_bq_utils as bqu
sys.path.pop()

import time

In [2]:
table_name = 'daily_opchain_aggregate_stats_dune'

In [3]:
# Run Dune
print('     dune runs')
dune_df = d.get_dune_data(query_id = 2453515, #https://dune.com/queries/2453515
    name = "dune_" + table_name,
    path = "outputs",
    performance="large"
)
dune_df['source'] = 'dune'
dune_df['dt'] = pd.to_datetime(dune_df['dt']).dt.tz_localize(None)


2024-06-02 20:08:54.416 | INFO     | duneapi_utils:get_dune_data:44 - Results available at https://dune.com/queries/2453515


     dune runs


2024-06-02 20:08:57.830 | INFO     | duneapi_utils:get_dune_data:89 - ✨ Results saved as outputs/dune_daily_opchain_aggregate_stats_dune.csv, with 788 rows and 348 columns.


In [4]:
# Replace inf, -inf, and NaN with 0
dune_df = dune_df.replace([np.inf, -np.inf, np.nan], 0)
dune_df = dune_df.replace(['inf', 'NaN'], 0)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_67244/3583440690.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dune_df = dune_df.replace([np.inf, -np.inf, np.nan], 0)


In [5]:
dune_df["avg_l1_calldata_gas_price_on_l1_inbox"] = dune_df["avg_l1_calldata_gas_price_on_l1_inbox"].astype('float64')
dune_df["avg_l1_calldata_gas_price_on_l1_inbox_by_day"] = dune_df["avg_l1_calldata_gas_price_on_l1_inbox_by_day"].astype('float64')
dune_df["source"] = dune_df["source"].astype('string')
dune_df["chain_layer"] = dune_df["chain_layer"].astype('string')
dune_df["chain_type"] = dune_df["chain_type"].astype('string')

# # dune_df.dtypes
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     # display(dune_df.sample(5))
#     print("\nData types:")
#     print(dune_df.dtypes[40:])

# print(dune_df["chain_layer"].unique())
# print(dune_df["chain_layer"].dtypes)

In [6]:
#BQ Upload
bqu.write_df_to_bq_table(dune_df, table_name)

True
Running locally
